# Introduction to Bigraphs

Bigraphs are great

In [1]:
import os
os.chdir('../bigraph')

In [4]:
from bigraph import Bigraph, Control, Node, Edge, Parallel, Merge, Big, InGroup, Condition, Reaction, Range, Assign, Init, Param, RuleGroup, Rules, Preds, System, BigraphicalReactiveSystem
from parse import parse_expression

## syntax

The bigraph syntax is as follows

In [6]:
sample = 'A.(B | C{p})'
parse_expression(sample)

A.(B | C{p})

TODO: show rendered results from running the system

TODO: render bigraphs in a nested way